# Monthly Daycon Flight Delay Prediction AI Competition 

## data pre-processing

#### function import

In [2]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import tensorflow as tf

from transformers import pipeline, set_seed, BertTokenizer, TFBertForSequenceClassification

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import re
import nltk 
from nltk.corpus import stopwords
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ


#### Load Data

In [3]:
### 학습용 데이터 불러오기

file_path = 'D:\Code\월간 데이콘 항공편 지연 예측 AI 경진대회/train.csv'

df_train = pd.read_csv(file_path)

print(f'학습용 데이터 : \n{df_train}')

print('*'*80)

### 평가용 데이터 불러오기

file_path = 'D:\Code\월간 데이콘 항공편 지연 예측 AI 경진대회/test.csv'

df_test = pd.read_csv(file_path)

print(f'평가용 데이터 : \n{df_test}')

학습용 데이터 : 
                  ID  Month  Day_of_Month  Estimated_Departure_Time  \
0       TRAIN_000000      4            15                       NaN   
1       TRAIN_000001      8            15                     740.0   
2       TRAIN_000002      9             6                    1610.0   
3       TRAIN_000003      7            10                     905.0   
4       TRAIN_000004      1            11                     900.0   
...              ...    ...           ...                       ...   
999995  TRAIN_999995      9            18                     936.0   
999996  TRAIN_999996      5            30                     920.0   
999997  TRAIN_999997      6            28                     800.0   
999998  TRAIN_999998      9            27                    1613.0   
999999  TRAIN_999999      3            26                    1800.0   

        Estimated_Arrival_Time  Cancelled  Diverted Origin_Airport  \
0                          NaN          0         0            OKC

#### 컬럼의 설명 주석
- 'Estimated_Departure_Time' : 예상 출발 시간
- 'Estimated_Arrival_Time' : 예상 도착 시간
- 'Diverted' : 회항(방향전환)

In [5]:
columns = df_train.columns

print(columns) 


Index(['ID', 'Month', 'Day_of_Month', 'Estimated_Departure_Time',
       'Estimated_Arrival_Time', 'Cancelled', 'Diverted', 'Origin_Airport',
       'Origin_Airport_ID', 'Origin_State', 'Destination_Airport',
       'Destination_Airport_ID', 'Destination_State', 'Distance', 'Airline',
       'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number', 'Delay'],
      dtype='object')


#### 학습용 데이터의 결측치 확인 및 제거

In [13]:
n_nulls = df_train.isnull().sum()

print(n_nulls)

# 8개의 컬럼을 제거

print(df_train)

print('*'*80)

cleaned_df_train = df_train.drop(columns=['Estimated_Departure_Time','Estimated_Arrival_Time','Origin_State','Destination_State','Airline','Carrier_Code(IATA)','Carrier_ID(DOT)','Delay'])

print(cleaned_df_train)

# 1. df_train.dropna(subset=[
## subset의 axis는 어치파 인 지워도 됨. 누락값을 지울거니까. 컬럼만 지정해줄 때 쓰는 subset.
# 2. Delay가 정답컬럼과 연관이 있는지..? 

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64
                  ID  Month  Day_of_Month  Estimated_Departure_Time  \
0       TRAIN_000000      4            15                       NaN   
1       TRAIN_000001      8            15                     740.0   
2       TRAIN_000002      9             6                    1610.0   
3       TRAIN_000003      7           